In [150]:
using BenchmarkTools

In [143]:
module sp

import Base: *, +, promote_type, promote_op, promote_rule, convert

immutable Variable{Name}
end

immutable Monomial{N, V}
    exponents::NTuple{N, Int64}
end

variables(::Type{Monomial{N, V}}) where {N, V} = V

Monomial(v::Variable{Name}) where Name = Monomial{1, (Name,)}((1,))

*(v1::Variable{N1}, v2::Variable{N2}) where {N1, N2} = Monomial{2, (N1, N2)}((1, 1))
*(v1::Variable{N}, v2::Variable{N}) where N = Monomial{1, (N,)}((2,))

immutable Term{T, MonomialType <: Monomial}
    coefficient::T
    monomial::MonomialType
end

Term(m::Monomial) = Term(1, m)
Term(v::Variable) = Term(Monomial(v))
*(x::Number, m::Monomial) = Term(x, m)
@generated function convert(::Type{Term{T, Mono1}}, t::Term{T, Mono2}) where {T, Mono1, Mono2}
    args = Any[0 for v in variables(Mono1)]
    for (j, var) in enumerate(variables(Mono2))
        I = find(v -> v == var, variables(Mono1))
        if isempty(I)
            throw(InexactError())
        elseif length(I) > 1
            error("Duplicate variables in destination $Mono1")
        end
        args[I[1]] = :(t.monomial.exponents[$j])
    end
    quote
        Term{$T, $Mono1}(t.coefficient, 
            Monomial{$(length(args)), $(variables(Mono1))}($(Expr(:tuple, args...))))
    end
end
    
        
immutable Polynomial{M, TermType <: Term}
    terms::NTuple{M, TermType}
end

@generated function promote_rule(::Type{Term{T, Mono1}}, ::Type{Term{T, Mono2}}) where {T, Mono1, Mono2}
    vars = Set{Symbol}()
    for v in variables(Mono1)
        push!(vars, v)
    end
    for v in variables(Mono2)
        push!(vars, v)
    end
    vars = Tuple(sort(collect(vars)))
    quote
        Term{T, Monomial{$(length(vars)), $(vars)}}
    end
end

(+)(t1::Term{T, Mono}, t2::Term{T, Mono}) where {T, Mono} = Polynomial((t1, t2))
(+)(t1::Term{T, Mono1}, t2::Term{T, Mono2}) where {T, Mono1, Mono2} = +(promote(t1, t2)...)
(+)(m1::Monomial, m2::Monomial) = Term(m1) + Term(m2)
(+)(v1::Variable, v2::Variable) = Term(v1) + Term(v2)

end

IJulia.sp

In [144]:
x = sp.Variable{:x}()
y = sp.Variable{:y}()

tx = sp.Term(x)
ty = sp.Term(y)

IJulia.sp.Term{Int64,IJulia.sp.Monomial{1,(:y,)}}(1, IJulia.sp.Monomial{1,(:y,)}((1,)))

In [152]:
@benchmark $x + $y

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.253 ns (0.00% GC)
  median time:      2.462 ns (0.00% GC)
  mean time:        3.113 ns (0.00% GC)
  maximum time:     47.258 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [105]:
Base.promote_type(typeof(sp.Term(x)), typeof(sp.Term(y)))

IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:y, :x)}}

In [106]:
x + y

LoadError: [91mMethodError: no method matching +(::IJulia.sp.Term{Int64,IJulia.sp.Monomial{1,(:x,)}}, ::IJulia.sp.Term{Int64,IJulia.sp.Monomial{1,(:y,)}})[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +(::IJulia.sp.Term{T,Mono}, [91m::IJulia.sp.Term{T,Mono}[39m) where {T, Mono} at In[102]:45[39m

In [67]:
tuple((1,2,3))

((1, 2, 3),)

In [52]:
x * y

IJulia.sp.Monomial{2,(:x, :y)}((1, 1))

In [53]:
5 * (x * y)

IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}}(5, IJulia.sp.Monomial{2,(:x, :y)}((1, 1)))

In [54]:
x * x

IJulia.sp.Monomial{1,(:x,)}((2,))

In [48]:
5 * (x * y) + 3 * (x * y)

IJulia.sp.Polynomial{2,IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}}}((IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}}(5, IJulia.sp.Monomial{2,(:x, :y)}((1, 1))), IJulia.sp.Term{Int64,IJulia.sp.Monomial{2,(:x, :y)}}(3, IJulia.sp.Monomial{2,(:x, :y)}((1, 1)))))

In [18]:
sp.Monomial{2, (:x, :y)}

IJulia.sp.Monomial{2,(:x, :y)}